In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Input, InputLayer, Dropout, Dense, Flatten, Embedding, Add, Concatenate
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import class_weight

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop

import matplotlib.pyplot as plt

from sklearn.metrics import precision_recall_fscore_support

## Own code 
import import_ipynb
import metadata_options
import models_nn

importing Jupyter notebook from metadata_options.ipynb
importing Jupyter notebook from models_nn.ipynb


In [2]:
# Sound for when code starts and ends
import os
duration = 1  # seconds
freq = 600  # Hz
os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq))

0

In [3]:
def create_random_text(dataframe, input_text_col_name, num_words_col_name, size_of_output, seed):
    np.random.seed(seed)
    
    min_num_words = dataframe[num_words_col_name].min()
    max_num_words = dataframe[num_words_col_name].max()
    
    def choose_random_word(dataframe, input_text_col_name, is_start, is_end):
        random_index = np.random.randint(0, dataframe.shape[0] - 1)
        df_random_sentence = dataframe[input_text_col_name].iloc[random_index]
        while df_random_sentence == "#NAME?":
            random_index = np.random.randint(0, dataframe.shape[0] - 1)
            df_random_sentence = dataframe[input_text_col_name].iloc[random_index]
        if is_start:
            return df_random_sentence.split(' ')[0]  
        if is_end:
            return df_random_sentence.split(' ')[-1]
        index_words = df_random_sentence.split(' ')[1:-1]
        return np.random.choice(index_words)
    
    store_sentences = []
    for i in range(size_of_output):
        sentence_num_words = np.random.randint(min_num_words, max_num_words)
        sentence = choose_random_word(dataframe, input_text_col_name, True, False) # Choose first word
        for j in range(sentence_num_words - 1 - 1): # -1 for first word already chosen, -1 for ending word
            sentence += ' ' + choose_random_word(dataframe, input_text_col_name, False, False)
        sentence += ' ' + choose_random_word(dataframe, input_text_col_name, False, True) #Choose last word
        store_sentences.append(sentence)
    
    return pd.DataFrame({input_text_col_name: store_sentences})

def create_stratified_label_df_column(original_column, column_name, out_df_length):
    df = original_column.copy()
    return df.groupby(column_name, group_keys=False).apply(lambda x: x.sample(int(np.rint(out_df_length*len(x)/len(df))))).sample(frac=1).reset_index(drop=True)

def create_metadata_df_column(original_column, meta_name, out_df_length):
    df = original_column.copy()
    df = df.squeeze()
    test = pd.DataFrame(np.random.choice(df,size=(out_df_length, 1)), columns=[meta_name])
    return test
    
def create_random_df(dataframe, input_text_col_name, num_words_col_name, size_of_output, seed, label_columns_list, meta_data_columns_list):
    # Start with input text column
    random_df = create_random_text(dataframe, input_text_col_name, num_words_col_name, size_of_output, seed)
    
    # Add the columns of label columns
    for label in label_columns_list:
        label_column = create_stratified_label_df_column(dataframe[label].to_frame(), label, size_of_output)
        random_df = random_df.join(label_column)
    
    # Add the columns of metadata columns
    for meta in meta_data_columns_list:
        meta_column = create_metadata_df_column(dataframe[meta].to_frame(), meta, size_of_output)
        random_df = random_df.join(meta_column)
    return random_df

In [4]:
# Data with Throughput & WorkTime
df = pd.read_csv('./data/kokil dec 6 reprepare/conf_pc_worker_sem.csv')
df = df.dropna()

In [5]:
# Set if want use random_df or not
is_random = True
random_df_size = df.shape[0]
seed = 160121
columns_list = ['Input.deception_quadrant', 'Answer.3rapport.yes_label',
                'Answer.4shareinformation.yes_label', 'Answer.2reasoning.yes_label',
                'Answer.1gamemove.yes_label', ]
meta_list = ['Throughput.1', 'Throughput.2', 'Throughput.3', 'Throughput.4', 'Throughput.5',
             'WorkTime.1', 'WorkTime.2', 'WorkTime.3', 'WorkTime.4', 'WorkTime.5',
             'Answer.1gamemove.yes_pc_agree', 'Answer.2reasoning.yes_pc_agree', 'Answer.4shareinformation.yes_pc_agree', 'Answer.3rapport.yes_pc_agree',
             'Input.num_words', 'Input.num_characters']

if is_random:
    random_df = create_random_df(df, 'Input.full_text', 'Input.num_words', random_df_size, seed, columns_list, meta_list)
    drop_list = ['Input.full_text', 'Input.deception_quadrant', 'Answer.3rapport.yes_label', 'Answer.4shareinformation.yes_label', 'Answer.2reasoning.yes_label', 'Answer.1gamemove.yes_label']
    random_df = random_df.drop(drop_list, axis=1)
    list_to_replace = list(random_df)
    df.drop(labels=list_to_replace, axis="columns", inplace=True)
    df[list_to_replace] = random_df[list_to_replace]

In [6]:
######################################
## Weighted Onehot Encoding options ##
######################################

##############
# Throughput #
##############
# TP1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# TP2: weighted by 1 linear variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# TP3 + k: weighted by 1 inverted k-power U-shaped variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# TP4 + k: weighted by 1 upright k-power U-shaped variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# (For TP3 & TP4, k=1 results in V-shaped variance, and as k>1 increases, sides will curve into U-shaped variance)

############
# Worktime #
############
# WT1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# WT2: weighted by 1 linear variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)

################
# PC agreement #
################
# PC1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# PC2: weighted by 1 linear variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# PC3: weighted by 1 PC agreement weight per annotation in each OHE, i.e. (a, b, c, d) -> (w1*a, w2*b, w3*c, w4*d)

#####################
# Input text length #
#####################
# TL1: weighted by 1 normalised number of characters per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# TL2: weighted by 1 normalised number of words per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)

###################
# Special Options #
###################
# SP1: weighted by average of TP1 and TP2 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# SP2: weighted by average of WT1 and WT2 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# SP3: weighted by average of PC1 and PC2 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# RAND_UNI: weighted by 1 uniformly distributed random number between 0 to 1 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# RAND_NORM: weighted by 1 normally distributed random number between 0 to 1 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)

# Select 1 option from each of the few variants above, e.g. TP2, WT1, PC3, TL1, SP3, and input into function
# set_OHE_pipeline_options. If not selecting TP3 or TP4, input k (option_k) will be ignored. After
# editing the options, run the entire notebook for results accordingly.

# Edit option choices here
throughput_option = 'TP1'
worktime_option = 'WT2'
pc_agreement_option = 'PC3'
textlength_option = 'TL2'
special_option = 'SP3'
k_option_for_tp = 3

df_throughput, df_worktime, df_agreement, df_textlength, df_special = metadata_options.set_OHE_pipeline_options(df, throughput_option, worktime_option, pc_agreement_option, textlength_option, special_option, k_option_for_tp)

TP1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
WT2: weighted by 1 linear variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
PC3: weighted by 1 PC agreement weight per annotation in each OHE, i.e. (a, b, c, d) -> (w1*a, w2*b, w3*c, w4*d)
TL2: weighted by 1 normalised number of words per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
SP3: weighted by average of PC1 and PC2 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)


In [7]:
######################################
## Model Options ##
######################################
# options: lstm, cnn, lstm-attn, bilstm

model_name = 'lstm'
models_nn.MODEL_NAME = model_name

In [8]:
# # Old train_test_split code
# train, test, indices_train, indices_test = train_test_split(df, indices, test_size=0.2)

# New train_test_split using Stratified Shaffled Splits
y = df["Input.deception_quadrant"].copy()
X = df.drop(["Input.deception_quadrant"], axis=1)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
splits_generator = sss.split(X, y)

for train_idx, test_idx in splits_generator:
    indices_train = train_idx
    indices_test = test_idx

train = df.take(indices_train)
test = df.take(indices_test)

In [9]:
print(df.shape, train.shape, test.shape)

(11366, 863) (9092, 863) (2274, 863)


In [10]:
# check scikit-learn version
import sklearn
print('sklearn: %s' % sklearn.__version__)

sklearn: 0.23.2


In [11]:
new_deception_train = train["Input.deception_quadrant"].copy()
new_deception_train['Input.deception_quadrant'] = train["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y_train_deception = new_deception_train['Input.deception_quadrant'].to_numpy()
deception_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_deception),
                                                 y_train_deception)
deception_class_weight_dict = dict(enumerate(deception_class_weights))

y_train_rapport = train['Answer.3rapport.yes_label'].tolist()
rapport_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_rapport),
                                                 y_train_rapport)
rapport_class_weight_dict = dict(enumerate(rapport_class_weights))

y_train_share_information = train['Answer.4shareinformation.yes_label'].tolist()
share_info_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_share_information),
                                                 y_train_share_information)
share_info_class_weight_dict = dict(enumerate(share_info_class_weights))

y_train_reasoning = train['Answer.2reasoning.yes_label'].tolist()
reasoning_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_reasoning),
                                                 y_train_reasoning)
reasoning_class_weight_dict = dict(enumerate(reasoning_class_weights))

y_train_gamemove = train['Answer.1gamemove.yes_label'].tolist()
gamemove_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_gamemove),
                                                 y_train_gamemove)
gamemove_class_weight_dict = dict(enumerate(gamemove_class_weights))

/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1], y=[1 1 1 ... 1 1 1] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0. 1.], y=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 

In [12]:
print("Class weights")
print("Deception: {} \nRapport: {} \nShare Information: {} \nReasoning: {} \nGamemove: {}".format(deception_class_weight_dict,
                                                                                                  rapport_class_weight_dict,
                                                                                                  share_info_class_weight_dict,
                                                                                                  reasoning_class_weight_dict,
                                                                                                  gamemove_class_weight_dict))

Class weights
Deception: {0: 10.192825112107624, 1: 0.525792273883877} 
Rapport: {0: 3.582348305752561, 1: 0.5811069922024799} 
Share Information: {0: 3.117969821673525, 1: 0.59549384333246} 
Reasoning: {0: 2.9596354166666665, 1: 0.6016410799364743} 
Gamemove: {0: 7.048062015503876, 1: 0.5381792352314431}


In [13]:
X_train_col = train['Input.full_text']

new_deception_test = test["Input.deception_quadrant"].copy()
new_deception_test['Input.deception_quadrant'] = test["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y_test_deception = new_deception_test['Input.deception_quadrant'].tolist()
y_test_rapport = test['Answer.3rapport.yes_label'].tolist()
y_test_share_information = test['Answer.4shareinformation.yes_label'].tolist()
y_test_reasoning = test['Answer.2reasoning.yes_label'].tolist()
y_test_gamemove = test['Answer.1gamemove.yes_label'].tolist()

X_test_col = test['Input.full_text']

In [14]:
le = LabelEncoder()

# def label_preprocessing(y_data, label_encoder):
#     out = label_encoder.fit_transform(y_data).reshape(-1,1)
#     return out

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_train_rapport = le.fit_transform(y_train_rapport)
y_train_rapport = y_train_rapport.reshape(-1,1)

y_train_share_information = le.fit_transform(y_train_share_information)
y_train_share_information = y_train_share_information.reshape(-1,1)

y_train_reasoning = le.fit_transform(y_train_reasoning)
y_train_reasoning = y_train_reasoning.reshape(-1,1)

y_train_gamemove = le.fit_transform(y_train_gamemove)
y_train_gamemove = y_train_gamemove.reshape(-1,1)

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_test_rapport = le.fit_transform(y_test_rapport)
y_test_rapport = y_test_rapport.reshape(-1,1)

y_test_share_information = le.fit_transform(y_test_share_information)
y_test_share_information = y_test_share_information.reshape(-1,1)

y_test_reasoning = le.fit_transform(y_test_reasoning)
y_test_reasoning = y_test_reasoning.reshape(-1,1)

y_test_gamemove = le.fit_transform(y_test_gamemove)
y_test_gamemove = y_test_gamemove.reshape(-1,1)

y_test_deception = le.fit_transform(y_test_deception)
y_test_deception = y_test_deception.reshape(-1,1)

y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [15]:
max_words = 1000
max_len = 220

tok = Tokenizer(num_words=max_words)

tok.fit_on_texts(X_train_col)
X_train_sequences = tok.texts_to_sequences(X_train_col)
X_train = pad_sequences(X_train_sequences, maxlen=max_len)

X_test_sequences = tok.texts_to_sequences(X_test_col)
X_test = pad_sequences(X_test_sequences, maxlen=max_len)

### Construct individual  models

In [16]:
# Rapport model
rapport_model = models_nn.create_nn_model()
rapport_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, 
                                                                              models_nn.recall_m, models_nn.precision_m])
rapport_model.fit(X_train,y_train_rapport,
                  batch_size=128,
                  epochs=50,
                  validation_data=(X_test, y_test_rapport), 
#                   callbacks=[models_nn.early_stop],
                  class_weight=rapport_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 2s 24ms/step - loss: 0.6935 - accuracy: 0.5256 - f1_m: 0.5543 - recall_m: 0.5415 - precision_m: 0.8112 - val_loss: 0.6336 - val_accuracy: 0.8712 - val_f1_m: 0.9305 - val_recall_m: 1.0000 - val_precision_m: 0.8704
Epoch 2/50
72/72 [==============================] - 1s 20ms/step - loss: 0.6882 - accuracy: 0.5907 - f1_m: 0.6930 - recall_m: 0.6167 - precision_m: 0.8828 - val_loss: 0.6386 - val_accuracy: 0.7010 - val_f1_m: 0.8175 - val_recall_m: 0.7719 - val_precision_m: 0.8700
Epoch 3/50
72/72 [==============================] - 1s 19ms/step - loss: 0.6720 - accuracy: 0.6219 - f1_m: 0.7220 - recall_m: 0.6248 - precision_m: 0.8870 - val_loss: 0.8473 - val_accuracy: 0.2555 - val_f1_m: 0.2894 - val_recall_m: 0.1759 - val_precision_m: 0.8556
Epoch 4/50
72/72 [==============================] - 1s 19ms/step - loss: 0.6492 - accuracy: 0.6021 - f1_m: 0.7161 - recall_m: 0.5981 - precision_m: 0.9196 - val_loss: 0.5801 - val_accuracy: 0.7265 - val_f1

Epoch 32/50
72/72 [==============================] - 1s 19ms/step - loss: 0.3204 - accuracy: 0.8059 - f1_m: 0.8703 - recall_m: 0.7814 - precision_m: 0.9873 - val_loss: 1.3235 - val_accuracy: 0.6016 - val_f1_m: 0.7356 - val_recall_m: 0.6416 - val_precision_m: 0.8644
Epoch 33/50
72/72 [==============================] - 1s 19ms/step - loss: 0.3125 - accuracy: 0.8064 - f1_m: 0.8727 - recall_m: 0.7846 - precision_m: 0.9881 - val_loss: 1.1997 - val_accuracy: 0.6535 - val_f1_m: 0.7789 - val_recall_m: 0.7090 - val_precision_m: 0.8666
Epoch 34/50
72/72 [==============================] - 1s 20ms/step - loss: 0.3137 - accuracy: 0.8088 - f1_m: 0.8763 - recall_m: 0.7912 - precision_m: 0.9873 - val_loss: 1.1935 - val_accuracy: 0.6566 - val_f1_m: 0.7823 - val_recall_m: 0.7137 - val_precision_m: 0.8680
Epoch 35/50
72/72 [==============================] - 1s 19ms/step - loss: 0.3085 - accuracy: 0.8118 - f1_m: 0.8783 - recall_m: 0.7939 - precision_m: 0.9880 - val_loss: 1.7903 - val_accuracy: 0.4301 - va

In [17]:
rapport_pred = rapport_model.predict(X_train)
rapport_pred_test = rapport_model.predict(X_test)

rapport_pred_test_round = rapport_pred_test.round()
precision_recall_fscore_support(y_test_rapport, rapport_pred_test_round, average='macro')

(0.49164594200771466, 0.48225032001970947, 0.44452180054231755, None)

In [18]:
# Game move model
gamemove_model = models_nn.create_nn_model()
gamemove_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, 
                                                                               models_nn.recall_m, models_nn.precision_m])
gamemove_model.fit(X_train,
                   y_train_gamemove,
                   batch_size=128,
                   epochs=50,
                   validation_data=(X_test, y_test_gamemove), 
#                    callbacks=[models_nn.early_stop],
                   class_weight=gamemove_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 2s 23ms/step - loss: 0.6927 - accuracy: 0.5921 - f1_m: 0.6534 - recall_m: 0.6031 - precision_m: 0.8566 - val_loss: 0.7478 - val_accuracy: 0.0976 - val_f1_m: 0.0771 - val_recall_m: 0.0405 - val_precision_m: 0.9544
Epoch 2/50
72/72 [==============================] - 1s 19ms/step - loss: 0.6842 - accuracy: 0.5643 - f1_m: 0.6758 - recall_m: 0.5679 - precision_m: 0.9497 - val_loss: 0.5740 - val_accuracy: 0.8417 - val_f1_m: 0.9132 - val_recall_m: 0.8880 - val_precision_m: 0.9408
Epoch 3/50
72/72 [==============================] - 1s 20ms/step - loss: 0.6459 - accuracy: 0.6665 - f1_m: 0.7774 - recall_m: 0.6730 - precision_m: 0.9589 - val_loss: 0.4753 - val_accuracy: 0.8210 - val_f1_m: 0.9003 - val_recall_m: 0.8652 - val_precision_m: 0.9395
Epoch 4/50
72/72 [==============================] - 1s 19ms/step - loss: 0.5952 - accuracy: 0.6737 - f1_m: 0.7862 - recall_m: 0.6709 - precision_m: 0.9678 - val_loss: 0.4932 - val_accuracy: 0.7551 - val_f1

Epoch 32/50
72/72 [==============================] - 2s 21ms/step - loss: 0.2629 - accuracy: 0.8518 - f1_m: 0.9133 - recall_m: 0.8471 - precision_m: 0.9949 - val_loss: 0.9301 - val_accuracy: 0.7221 - val_f1_m: 0.8356 - val_recall_m: 0.7548 - val_precision_m: 0.9375
Epoch 33/50
72/72 [==============================] - 1s 18ms/step - loss: 0.2520 - accuracy: 0.8544 - f1_m: 0.9147 - recall_m: 0.8499 - precision_m: 0.9951 - val_loss: 0.9859 - val_accuracy: 0.7326 - val_f1_m: 0.8431 - val_recall_m: 0.7683 - val_precision_m: 0.9353
Epoch 34/50
72/72 [==============================] - 1s 19ms/step - loss: 0.2575 - accuracy: 0.8507 - f1_m: 0.9123 - recall_m: 0.8452 - precision_m: 0.9958 - val_loss: 0.9716 - val_accuracy: 0.7274 - val_f1_m: 0.8391 - val_recall_m: 0.7599 - val_precision_m: 0.9379
Epoch 35/50
72/72 [==============================] - 1s 19ms/step - loss: 0.2362 - accuracy: 0.8707 - f1_m: 0.9240 - recall_m: 0.8663 - precision_m: 0.9955 - val_loss: 1.0951 - val_accuracy: 0.7577 - va

In [19]:
gamemove_pred = gamemove_model.predict(X_train)
gamemove_pred_test = gamemove_model.predict(X_test)

gamemove_pred_test_round = gamemove_pred_test.round()
precision_recall_fscore_support(y_test_gamemove, gamemove_pred_test_round, average='macro')

(0.5009227809390119, 0.5019455381444389, 0.49220414679703234, None)

In [20]:
# Reasoning model
reasoning_model = models_nn.create_nn_model()
reasoning_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, models_nn.recall_m, models_nn.precision_m])
reasoning_model.fit(X_train,
                    y_train_reasoning,
                    batch_size=128,
                    epochs=50,
                    validation_data=(X_test, y_test_reasoning), 
#                     callbacks=[models_nn.early_stop],
                    class_weight=reasoning_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 2s 25ms/step - loss: 0.6933 - accuracy: 0.5250 - f1_m: 0.5990 - recall_m: 0.5472 - precision_m: 0.8069 - val_loss: 0.6715 - val_accuracy: 0.8219 - val_f1_m: 0.9011 - val_recall_m: 1.0000 - val_precision_m: 0.8212
Epoch 2/50
72/72 [==============================] - 1s 20ms/step - loss: 0.6901 - accuracy: 0.6816 - f1_m: 0.7679 - recall_m: 0.7435 - precision_m: 0.8409 - val_loss: 1.0219 - val_accuracy: 0.1803 - val_f1_m: 0.0083 - val_recall_m: 0.0042 - val_precision_m: 0.2778
Epoch 3/50
72/72 [==============================] - 1s 19ms/step - loss: 0.6778 - accuracy: 0.5734 - f1_m: 0.6668 - recall_m: 0.5654 - precision_m: 0.8669 - val_loss: 0.7066 - val_accuracy: 0.4727 - val_f1_m: 0.5815 - val_recall_m: 0.4501 - val_precision_m: 0.8289
Epoch 4/50
72/72 [==============================] - 1s 20ms/step - loss: 0.6592 - accuracy: 0.5969 - f1_m: 0.7013 - recall_m: 0.5872 - precision_m: 0.8911 - val_loss: 0.5949 - val_accuracy: 0.7062 - val_f1

Epoch 32/50
72/72 [==============================] - 1s 19ms/step - loss: 0.3352 - accuracy: 0.7811 - f1_m: 0.8494 - recall_m: 0.7502 - precision_m: 0.9835 - val_loss: 1.5705 - val_accuracy: 0.6310 - val_f1_m: 0.7519 - val_recall_m: 0.6871 - val_precision_m: 0.8328
Epoch 33/50
72/72 [==============================] - 2s 23ms/step - loss: 0.3323 - accuracy: 0.7834 - f1_m: 0.8529 - recall_m: 0.7569 - precision_m: 0.9827 - val_loss: 1.5497 - val_accuracy: 0.5523 - val_f1_m: 0.6766 - val_recall_m: 0.5734 - val_precision_m: 0.8284
Epoch 34/50
72/72 [==============================] - 1s 19ms/step - loss: 0.3263 - accuracy: 0.7839 - f1_m: 0.8505 - recall_m: 0.7533 - precision_m: 0.9832 - val_loss: 1.2519 - val_accuracy: 0.5695 - val_f1_m: 0.6937 - val_recall_m: 0.5977 - val_precision_m: 0.8294
Epoch 35/50
72/72 [==============================] - 1s 21ms/step - loss: 0.3105 - accuracy: 0.8005 - f1_m: 0.8659 - recall_m: 0.7755 - precision_m: 0.9849 - val_loss: 2.2420 - val_accuracy: 0.4938 - va

In [21]:
reasoning_pred = reasoning_model.predict(X_train)
reasoning_pred_test = reasoning_model.predict(X_test)

reasoning_pred_test_round = reasoning_pred_test.round()
precision_recall_fscore_support(y_test_reasoning, reasoning_pred_test_round, average='macro')

(0.5057770627732563, 0.509175039137586, 0.4841372548207389, None)

In [22]:
# Share Information model
shareinfo_model = models_nn.create_nn_model()
shareinfo_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, models_nn.recall_m, models_nn.precision_m])
shareinfo_model.fit(X_train,
                    y_train_share_information,
                    batch_size=128,
                    epochs=50,
                    validation_data=(X_test, y_test_share_information), 
#                     callbacks=[models_nn.early_stop],
                    class_weight=share_info_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 2s 25ms/step - loss: 0.6936 - accuracy: 0.5250 - f1_m: 0.5909 - recall_m: 0.5533 - precision_m: 0.8001 - val_loss: 0.6825 - val_accuracy: 0.8448 - val_f1_m: 0.9149 - val_recall_m: 1.0000 - val_precision_m: 0.8439
Epoch 2/50
72/72 [==============================] - 2s 21ms/step - loss: 0.6905 - accuracy: 0.5662 - f1_m: 0.6463 - recall_m: 0.5899 - precision_m: 0.8434 - val_loss: 0.6056 - val_accuracy: 0.8074 - val_f1_m: 0.8917 - val_recall_m: 0.9465 - val_precision_m: 0.8439
Epoch 3/50
72/72 [==============================] - 1s 20ms/step - loss: 0.6736 - accuracy: 0.5922 - f1_m: 0.7013 - recall_m: 0.5978 - precision_m: 0.8818 - val_loss: 0.6061 - val_accuracy: 0.6680 - val_f1_m: 0.7892 - val_recall_m: 0.7439 - val_precision_m: 0.8439
Epoch 4/50
72/72 [==============================] - 1s 19ms/step - loss: 0.6497 - accuracy: 0.5860 - f1_m: 0.6898 - recall_m: 0.5667 - precision_m: 0.9027 - val_loss: 0.6084 - val_accuracy: 0.6526 - val_f1

Epoch 32/50
72/72 [==============================] - 2s 21ms/step - loss: 0.3430 - accuracy: 0.7833 - f1_m: 0.8558 - recall_m: 0.7667 - precision_m: 0.9734 - val_loss: 1.3228 - val_accuracy: 0.4943 - val_f1_m: 0.6162 - val_recall_m: 0.4833 - val_precision_m: 0.8549
Epoch 33/50
72/72 [==============================] - 1s 21ms/step - loss: 0.3454 - accuracy: 0.7810 - f1_m: 0.8533 - recall_m: 0.7626 - precision_m: 0.9746 - val_loss: 1.4772 - val_accuracy: 0.6020 - val_f1_m: 0.7322 - val_recall_m: 0.6475 - val_precision_m: 0.8448
Epoch 34/50
72/72 [==============================] - 1s 19ms/step - loss: 0.3310 - accuracy: 0.7970 - f1_m: 0.8642 - recall_m: 0.7822 - precision_m: 0.9714 - val_loss: 1.4636 - val_accuracy: 0.4780 - val_f1_m: 0.5927 - val_recall_m: 0.4527 - val_precision_m: 0.8634
Epoch 35/50
72/72 [==============================] - 1s 20ms/step - loss: 0.3401 - accuracy: 0.7919 - f1_m: 0.8621 - recall_m: 0.7778 - precision_m: 0.9723 - val_loss: 1.5030 - val_accuracy: 0.6838 - va

In [23]:
shareinfo_pred = shareinfo_model.predict(X_train)
shareinfo_pred_test = shareinfo_model.predict(X_test)

shareinfo_pred_test_round = shareinfo_pred_test.round()
precision_recall_fscore_support(y_test_share_information, shareinfo_pred_test_round, average='macro')

(0.5102160319551624, 0.5172492937853108, 0.49111519896860206, None)

In [24]:
# Deception model
deception_model = models_nn.create_nn_model()
deception_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, models_nn.recall_m, models_nn.precision_m])
deception_model.fit(X_train,
                    y_train_deception,
                    epochs=50,
                    validation_data=(X_test, y_test_deception), 
#                     callbacks=[models_nn.early_stop],
                    class_weight=deception_class_weight_dict)

Epoch 1/50
285/285 [==============================] - 5s 16ms/step - loss: 0.6948 - accuracy: 0.6881 - f1_m: 0.7387 - recall_m: 0.7059 - precision_m: 0.8548 - val_loss: 0.5913 - val_accuracy: 0.9507 - val_f1_m: 0.9747 - val_recall_m: 1.0000 - val_precision_m: 0.9514
Epoch 2/50
285/285 [==============================] - 4s 16ms/step - loss: 0.6849 - accuracy: 0.7411 - f1_m: 0.8151 - recall_m: 0.7610 - precision_m: 0.9606 - val_loss: 0.5968 - val_accuracy: 0.7449 - val_f1_m: 0.8515 - val_recall_m: 0.7703 - val_precision_m: 0.9575
Epoch 3/50
285/285 [==============================] - 4s 15ms/step - loss: 0.6605 - accuracy: 0.6556 - f1_m: 0.7672 - recall_m: 0.6598 - precision_m: 0.9709 - val_loss: 0.8023 - val_accuracy: 0.3417 - val_f1_m: 0.4808 - val_recall_m: 0.3272 - val_precision_m: 0.9661
Epoch 4/50
285/285 [==============================] - 4s 15ms/step - loss: 0.6393 - accuracy: 0.6344 - f1_m: 0.7516 - recall_m: 0.6339 - precision_m: 0.9722 - val_loss: 0.2954 - val_accuracy: 0.9230 

Epoch 32/50
285/285 [==============================] - 5s 16ms/step - loss: 0.3461 - accuracy: 0.8371 - f1_m: 0.9049 - recall_m: 0.8343 - precision_m: 0.9932 - val_loss: 0.5955 - val_accuracy: 0.7502 - val_f1_m: 0.8506 - val_recall_m: 0.7735 - val_precision_m: 0.9528
Epoch 33/50
285/285 [==============================] - 4s 16ms/step - loss: 0.3402 - accuracy: 0.8470 - f1_m: 0.9109 - recall_m: 0.8453 - precision_m: 0.9932 - val_loss: 0.6660 - val_accuracy: 0.6878 - val_f1_m: 0.8068 - val_recall_m: 0.7035 - val_precision_m: 0.9546
Epoch 34/50
285/285 [==============================] - 4s 15ms/step - loss: 0.3495 - accuracy: 0.8419 - f1_m: 0.9061 - recall_m: 0.8400 - precision_m: 0.9917 - val_loss: 0.6331 - val_accuracy: 0.7142 - val_f1_m: 0.8254 - val_recall_m: 0.7339 - val_precision_m: 0.9517
Epoch 35/50
285/285 [==============================] - 4s 15ms/step - loss: 0.3181 - accuracy: 0.8525 - f1_m: 0.9144 - recall_m: 0.8506 - precision_m: 0.9930 - val_loss: 0.6461 - val_accuracy: 0.7

In [25]:
deception_pred = deception_model.predict(X_train)
deception_pred_test = deception_model.predict(X_test)
deception_pred_test_round = deception_pred_test.round()
precision_recall_fscore_support(y_test_deception, deception_pred_test_round, average='macro')

(0.500481180052598, 0.5021557420377957, 0.446337208228457, None)

In [26]:
# deception_pred_test

In [27]:
# unique, counts = np.unique(deception_pred_test_round, return_counts=True)

In [28]:
# print(unique, counts)

In [29]:
# deception_model.history.history['val_f1_m']

### One hot encodings

In [30]:
# Train encodings
pred_df_arr_full = []
pred_df_arr = []
for i in range(0, len(gamemove_pred)):
    pred_obj_1 = {}
    pred_obj_1['gamemove'] = gamemove_pred[i][0]
    pred_obj_1['reasoning'] = reasoning_pred[i][0]
    pred_obj_1['shareinfo'] = shareinfo_pred[i][0]
    pred_df_arr.append(pred_obj_1)
    
    pred_obj_2 = pred_obj_1.copy()
    pred_obj_2['rapport'] = rapport_pred[i][0]
    pred_df_arr_full.append(pred_obj_2)
    
pred_df_full = pd.DataFrame(pred_df_arr_full)
pred_df = pd.DataFrame(pred_df_arr)

In [31]:
# Test encodings
pred_test_df_arr_full = []
pred_test_df_arr = []

for i in range(0, len(gamemove_pred_test)):
    pred_obj_1 = {}
    pred_obj_1['gamemove'] = gamemove_pred_test[i][0]
    pred_obj_1['reasoning'] = reasoning_pred_test[i][0]
    pred_obj_1['shareinfo'] = shareinfo_pred_test[i][0]
    pred_test_df_arr.append(pred_obj_1)
    
    pred_obj_2 = pred_obj_1.copy()
    pred_obj_2['rapport'] = rapport_pred_test[i][0]
    pred_test_df_arr_full.append(pred_obj_2)
    
pred_test_df_full = pd.DataFrame(pred_test_df_arr_full)
pred_test_df = pd.DataFrame(pred_test_df_arr)

### Joint Model with one hot encoding 

In [32]:
print('Joint full model with one hot encoding, predicting deception')
joint_full_model = models_nn.create_joint_model(pred_df_full)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_test_df_full,y_test_deception), 
#                                callbacks=[models_nn.callback], 
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception
Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 10        
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6980 - acc: 0.1980 - f1_m: 0.2622 - precision_m: 0.8889 - recall_m: 0.1634 - val_loss: 0.7073 - val_acc: 0.3610 - val_

In [33]:
joint_predict = joint_full_model.predict(pred_test_df_full)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.498049121585863, 0.4899068322981367, 0.4046197592611916, None)

In [34]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 937 1337]


In [35]:
print('Joint full model with one hot encoding, predicting rapport')
joint_full_model = models_nn.create_joint_model(pred_df)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_test_df,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport
Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 8         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6731 - acc: 0.8649 - f1_m: 0.9251 - precision_m: 0.8737 - recall_m: 0.9844 - val_loss: 0.6247 - val_acc: 0.8004 - val_f1

In [36]:
joint_predict = joint_full_model.predict(pred_test_df)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.4987955050131216, 0.49737868108808425, 0.44397485273580395, None)

In [37]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 965 1309]


## Weighted against Throughput, WorkTime, PC Agreement & Text Length

In [38]:
# Train weighted encodings
pred_df_full_throughput, pred_df_throughput, pred_df_full_worktime, pred_df_worktime, pred_df_full_agreement, pred_df_agreement, pred_df_full_textlength, pred_df_textlength, pred_df_full_special, pred_df_special = metadata_options.construct_weighted_dataframe(indices_train, df_throughput, df_worktime, df_agreement, df_textlength, df_special, pred_df, pred_df_full)

In [39]:
# Test weighted encodings
pred_df_full_throughput_test, pred_df_throughput_test, pred_df_full_worktime_test, pred_df_worktime_test, pred_df_full_agreement_test, pred_df_agreement_test, pred_df_full_textlength_test, pred_df_textlength_test, pred_df_full_special_test, pred_df_special_test = metadata_options.construct_weighted_dataframe(indices_test, df_throughput, df_worktime, df_agreement, df_textlength, df_special, pred_test_df, pred_test_df_full)

## Throughput only

### Joint Model with one hot encoding 

In [40]:
print('Joint full model with one hot encoding, predicting deception, weighted by throughput')
joint_full_model = models_nn.create_joint_model(pred_df_full_throughput)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_throughput, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_throughput_test,y_test_deception), 
#                                callbacks=[models_nn.callback], 
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by throughput
Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.8516 - acc: 0.9282 - f1_m: 0.9616 - precision_m: 0.9508 - recall_m: 0.9737 - val_loss: 0.4881 

In [41]:
joint_predict = joint_full_model.predict(pred_df_full_throughput_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.49465908692143035, 0.4717151777454738, 0.34662237705732174, None)

In [42]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [1239 1035]


In [43]:
y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

In [44]:
print('Joint full model with one hot encoding, predicting rapport, weighted by throughput')
joint_full_model = models_nn.create_joint_model(pred_df_throughput)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_throughput, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64,
                               validation_data=(pred_df_throughput_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by throughput
Model: "functional_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 8         
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6881 - acc: 0.6567 - f1_m: 0.6708 - precision_m: 0.6694 - recall_m: 0.6922 - val_loss: 0.7075 - 

In [45]:
joint_predict = joint_full_model.predict(pred_df_throughput_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.49520340286890807, 0.4897731521123024, 0.4477930835684525, None)

In [46]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 902 1372]


## WorkTime only

### Joint Model with one hot encoding 

In [47]:
print('Joint full model with one hot encoding, predicting deception, weighted by worktime')
joint_full_model = models_nn.create_joint_model(pred_df_full_worktime)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_worktime, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_worktime_test,y_test_deception), 
#                                callbacks=[models_nn.callback],
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by worktime
Model: "functional_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_13 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6932 - acc: 0.8128 - f1_m: 0.8301 - precision_m: 0.9210 - recall_m: 0.8427 - val_loss: 0.6935 - 

In [48]:
joint_predict = joint_full_model.predict(pred_df_full_worktime_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.5043152235427278, 0.5083132350997753, 0.5009254081860881, None)

In [49]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 228 2046]


In [50]:
print('Joint full model with one hot encoding, predicting rapport, weighted by worktime')
joint_full_model = models_nn.create_joint_model(pred_df_worktime)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_worktime, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64,
                               validation_data=(pred_df_worktime_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by worktime
Model: "functional_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_16 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6941 - acc: 0.7875 - f1_m: 0.8718 - precision_m: 0.8478 - recall_m: 0.9045 - val_loss: 0.6949 - va

143/143 [==============================] - 0s 2ms/step - loss: 0.6919 - acc: 0.2826 - f1_m: 0.3054 - precision_m: 0.8912 - recall_m: 0.1879 - val_loss: 0.6887 - val_acc: 0.3091 - val_f1_m: 0.3788 - val_precision_m: 0.8647 - val_recall_m: 0.2453
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6918 - acc: 0.3106 - f1_m: 0.3536 - precision_m: 0.8935 - recall_m: 0.2254 - val_loss: 0.6908 - val_acc: 0.2612 - val_f1_m: 0.2976 - val_precision_m: 0.8616 - val_recall_m: 0.1821
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6918 - acc: 0.5498 - f1_m: 0.6189 - precision_m: 0.8852 - recall_m: 0.5545 - val_loss: 0.6922 - val_acc: 0.2419 - val_f1_m: 0.2604 - val_precision_m: 0.8586 - val_recall_m: 0.1556
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6918 - acc: 0.2840 - f1_m: 0.3017 - precision_m: 0.8885 - recall_m: 0.1887 - val_loss: 0.6909 - val_acc: 0.2590 - val_f1_m: 0.2930 - val_precision_m: 0.8627 - val_r

In [51]:
joint_predict = joint_full_model.predict(pred_df_worktime_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.4970720601471912, 0.49618905196637686, 0.25724520741923196, None)

In [52]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [1870  404]


## PC Agreement only

### Joint Model with one hot encoding 

In [53]:
print('Joint full model with one hot encoding, predicting deception, weighted by PC Agreement')
joint_full_model = models_nn.create_joint_model(pred_df_full_agreement)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_agreement, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_agreement_test,y_test_deception), 
#                                callbacks=[models_nn.callback],
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by PC Agreement
Model: "functional_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_18 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_19 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6937 - acc: 0.6741 - f1_m: 0.7914 - precision_m: 0.9530 - recall_m: 0.6932 - val_loss: 0.685

In [54]:
joint_predict = joint_full_model.predict(pred_df_full_throughput_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.5014297526355681, 0.5031459272646456, 0.44064988782204645, None)

In [55]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [1012 1262]


In [56]:
y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

In [57]:
print('Joint full model with one hot encoding, predicting rapport, weighted by PC Agreement')
joint_full_model = models_nn.create_joint_model(pred_df_agreement)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_agreement, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_agreement_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by PC Agreement
Model: "functional_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_22 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6763 - acc: 0.8306 - f1_m: 0.9003 - precision_m: 0.8884 - recall_m: 0.9184 - val_loss: 0.6612 

In [58]:
joint_predict = joint_full_model.predict(pred_df_throughput_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.49016685968506957, 0.48117353768652027, 0.4631631487593644, None)

In [59]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 711 1563]


## Text Length only

### Joint Model with one hot encoding 

In [60]:
print('Joint full model with one hot encoding, predicting deception, weighted by text length')
joint_full_model = models_nn.create_joint_model(pred_df_full_textlength)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_textlength, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_textlength_test,y_test_deception), 
#                                callbacks=[models_nn.callback],
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by text length
Model: "functional_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_24 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_25 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6935 - acc: 0.8490 - f1_m: 0.8895 - precision_m: 0.9434 - recall_m: 0.8830 - val_loss: 0.6925

In [61]:
joint_predict = joint_full_model.predict(pred_df_full_textlength_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.5061810809474361, 0.5015486652570371, 0.05893140603785819, None)

In [62]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [2247   27]


In [63]:
print('Joint full model with one hot encoding, predicting rapport, weighted by text length')
joint_full_model = models_nn.create_joint_model(pred_df_textlength)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_textlength, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_textlength_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by text length
Model: "functional_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 3)]               0         
_________________________________________________________________
dense_27 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_28 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6932 - acc: 0.7694 - f1_m: 0.8543 - precision_m: 0.8483 - recall_m: 0.8775 - val_loss: 0.6924 -

In [64]:
joint_predict = joint_full_model.predict(pred_df_textlength_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.49581309451528033, 0.4914460411451451, 0.459602324837789, None)

In [65]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 810 1464]


## Other special options (either SP1, SP2, SP3, RAND_UNI, or RAND_NORM)

### Joint Model with one hot encoding 

In [66]:
print('Joint full model with one hot encoding, predicting deception, weighted by special option')
joint_full_model = models_nn.create_joint_model(pred_df_full_special)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_special, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_special_test,y_test_deception), 
#                                callbacks=[models_nn.callback],
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by special option
Model: "functional_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
dense_30 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_31 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6932 - acc: 0.6762 - f1_m: 0.7934 - precision_m: 0.9512 - recall_m: 0.6996 - val_loss: 0.6

In [67]:
joint_predict = joint_full_model.predict(pred_df_full_special_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.4984473232996761, 0.4946147746795295, 0.21334085701854424, None)

In [68]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [1810  464]


In [69]:
y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

In [70]:
print('Joint full model with one hot encoding, predicting rapport, weighted by special option')
joint_full_model = models_nn.create_joint_model(pred_df_special)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_special, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64,
                               validation_data=(pred_df_special_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by special option
Model: "functional_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 3)]               0         
_________________________________________________________________
dense_33 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_34 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_35 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6921 - acc: 0.3833 - f1_m: 0.3380 - precision_m: 0.3400 - recall_m: 0.3373 - val_loss: 0.695

In [71]:
joint_predict = joint_full_model.predict(pred_df_special_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.48547055744006423, 0.4720760191098714, 0.45678999207751875, None)

In [72]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 716 1558]


In [73]:
duration = 10  # seconds
freq = 600  # Hz
os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq))

0